# ПРЕДОБРАБОТКА ДАННЫХ
## Предсказание удельного электрического сопротивления (УЭС) пыли 

**Бизнес-постановка задачи** 

Для правильной разработки, проектирования, выбора типоразмера и изготовления установок электрогазоочистки, правильной пуско-наладки и эксплуатации электрофильтров (ЭФ) необходимо учитывать свойства улавливаемой пыли. А накопление большого слоя пыли вокруг электрочувствительных аппаратов может приводить к перебоям в их работе и аварийным ситуациям. 

**Постановка задачи анализа данных** 

Целью данной задачи является прогнозирование значения удельного электрического сопротивления пыли с помощью построения регрессионных моделей и их анализа. Зависимость УЭС пыли от температуры, влажности, дисперсного  и химического состава  пыли и газов  и от других параметров  используется в  расчете скорости дрейфа 
улавливаемых частиц.   Знание величины УЭС  пыли требуется  для  подготовки газов  перед электрофильтрами,   при определении  рациональных режимных параметров работы ЭФ для интенсификации процесса пылеулавливания.  

**Обзор доступных данных**

В выборке 360 наблюдений и 10 переменных. 
Таким образом, про каждое из 360 значений удельного электрического сопротивления мы знаем значения 9 его характеристик (температура, влажность, плотность пылевого слоя, размер частиц и т.д.)

## План анализа данных:

  1. Загрузить данные
  2. Обработать данные
  3. Построить регрессионную модель (классический вариант)
  4. Провести анализ полученной модели
  5. Разделить данные для обучения и тестирования (продвинутый вариант)
  6. Обучить модель на обучающей выборке
  7. Провалидировать модель на тестовой выборке

## 1. Загрузить данные

Библиотека **warnings** отвечает за то, какие предупреждения (warnings) о работе будут выводиться пользователю. 
FutureWarning - предупреждения о том, как изменится работа библиотек в будущих версиях.
Поэтому такие предупреждения мы будем игнорировать.
Чтобы включить режим игнорирования мы отбираем все предупреждения из категории FutureWarning и выбираем для них действия 'ignore'.
Это делается вызовом функции simplefilter c задание двух атрибутов: действия action и категории предупреждений category.

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

Для корректной работы с данными в python требуется загрузить специальную библиотеку
**pandas**, программную библиотеку на языке python для обработки и анализа данных.

In [ ]:
import pandas as pd # загружаем библиотеку и для простоты обращения в коде называем её сокращенно pd

Для корректной работы с графиками в python требуется загрузить специальную библиотеку
**matplotlib**, программную библиотеку на языке python для визуализации данных двумерной и трехмерной графикой.

Графики используются для облегчения интерпретации полученных результатов, а также в качестве иллюстраций в презентациях и отчетах. 

Оснвные методы для построения:
* plot() - графики
* semilogy() - график логарифмический
* hist() - гистограммы

In [ ]:
import matplotlib.pyplot as plt # загружаем библиотеку и для простоты обращения в коде называем её сокращенно plt
# указываем, чтобы картинки отображались прямо в ноутбуке 
%matplotlib inline 

Для решения задачи мы будем использовать данные. Так как данные сохранены в формате xlsx (Excel), мы будем использовать специальную функцию
из библиотеки pandas для загрузки таких данных **read_excel**.

In [ ]:
df = pd.read_excel('dust.xlsx') # загружаем таблицу в переменную df

*Что важно посмотреть после того, как мы загрузили данные?*
- проверить, что данные действительно загрузились
- посмотреть на данные, чтобы удостовериться, что они правильные: колонки имеют те же названия, что и в таблице и т.д.

Для того чтобы это сделать, нужно вызвать от переменной *df* метод **head()**, который выводит первые 5 строк таблицы. 

Для вызова метода объекта необходимо сначала написать *имя объекта*, затем поставить *точку*, затем уже написать *название метода*. Обратите внимание, что в конце обязательно ставить скобочки, потому что метод - это функция и в ней есть аргументы, просто в данном случае мы их не передаем, поэтому оставляем поле пустым

In [ ]:
df.head()

**Доступные признаки**

Данные содержат два типа переменных:

* Целевая: **resistivity**, УЭС пыли
* Остальные переменные: 9 переменных, могут использоваться для прогноза целевой переменной.

| Имя столбца        | Значение                                      | 
|:------------------:|:---------------------------------------------:|
| temperature        | температура образца пыли                      | 
| humidity           | влажность   образца пыли                      |
| density            | плотность образца                             |
| dust_capacity      | емкость пыли                                  | 
| particle_size      | размер частиц в образце                       | 
| is_chelyabinsk     | образец был взят на месторождении в Челябинске|
| conductivity       | электрическая проводимость                    |
| dust_dispersiveness| дисперсность пыли                             |
| formation          | способ образования пыли                       |
| resistivity        | **Целевая переменная:** УЭС пыли              |

Посмотрим на размеры загруженной таблицы, у которой мы видели только первые 5 строк.

Для этого вызываем поле **shape** у нашей переменной *df*. Поле вызывается также как метод, но в конце скобки не ставятся, так как для поля не предусмотрена передача аргументов. 

In [ ]:
df['resistivity'].value_counts()

In [ ]:
df.shape

*Что означает первое и второе число?*

Итак, таблица содержит 360 строк (объектов) и 10 столбцов (признаков), включая выходной (целевой) признак.
Таблицу проверили, теперь можно приступать к обработке данных.

# 2. Обработать данные

**Проверяем данные на наличие пропусков и типов переменных**

Начнем с проверки общей информации о данных.
Для того чтобы это сделать, нужно обратиться вызвать у переменной *df* метод **info()**.

Напомним, что в конце необходимо поставить скобочки.

In [ ]:
df.info()

Анализируем результата выполнения команды:

* 360 строк (entries)
* 10 столбцов (Data columns)

В данных присутствует три типа dtypes:
* int64 - целое число  (1 столбец)
* float64 - дробное число (6 столбцов)
* object - текстовые или смешанные числовые и нечисловые значения (3 столбца)



**Борьба с пропусками**

Цифры в каждой строчке обозначают количество заполненных (*non-null*) значений. 

Так как эти цифры не в каждой строчке совпадают с числом строк (360), то в данных имеются пропуски: это переменные `is_chelyabinsk` и `dust_capacity`.

С пропусками можно бороться несколькими способами:
- удаление пропусков
- заполнение пропусков

В переменной `is_chelyabinsk`, отвечающей за место получения образца пыли, порядка 30% пропущенных значений. Если предположить, что свойства пыли (химический состав) для различных образцов не отличаются, то мы можем избавиться от данной переменной.  

In [ ]:
df.drop(columns = ['is_chelyabinsk'], inplace = True)

Убедимся, что такого столбца больше нет при помощи метода **columns**.

In [ ]:
df.columns

Однако, удаление пропусков не всегда возможно, ведь это может привести к серьезной потере данных (как при удалении строк с отсутсвующими признаками, так и столбцов с пропусками в соответствующих признаках). В этом случае прибегают к заполнению пропусков. 

При заполнении используют простые или продвинутые методы. К простым можно отнести заполение некоторым константным значением или заполение средним/медианой.

Переменная `dust_capacity` содержить значение емкости пыли - это количественная характеристика, заполним имеющиеся пропуски (12 шт.) средним значением.

In [ ]:
mean = df.dust_capacity.mean()
mean

Теперь заполним пропуски в столбце `dust_capacity` средним значением емкости пыли.

In [ ]:
df.dust_capacity.fillna(mean, inplace = True)

Убедимся, что пропусков не осталось при помощи метода **isna**.

In [ ]:
df.dust_capacity.isna().sum()

In [ ]:
df.info()

**Визуализация**

Построим гистограммы данных.

**В чем польза таких графиков при анализе данных?**

In [ ]:
df.hist(figsize=(10, 10))

И тепловую карту корреляции признаков.

In [ ]:
import seaborn as sns
sns.heatmap(df.corr(), cbar_kws= {'orientation': 'vertical'}, annot=True, vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')

**Обработка текстовых переменных**

В наших данных 3 столбца  имеют значения типа object. В нашем случае, это текстовые признаки. Чтобы можно было подавать их на вход алгоритму или  модели, нам необходимо закодировать их.  

Мы рассмотрим два популярных метода кодирования категориальных признаков:
*  метод **get_dummies()** из библиотеки **pandas** (One-Hot encode)
*  метод **LabelEncoder()** из библиотеки **sklearn**

Для начала нам необходимо определить какие из признаков категориальные

In [ ]:
text_features = ['conductivity', 'dust_dispersiveness', 'formation']

Далее нам необходимо преобразовать значения в выбранных колонках к числам, потому что алгоритмы и модели не умеют работать с текстом.

Первый способ заключается в том, чтобы заменить признаки типа: 

|                 |    Электрическая проводимость |  
|-----------------|:-------------------------------:|
|**Образец пыли 1**  |       average                 | 
|**Образец пыли 2**  |       good                    | 
|**Образец пыли 3**  |       low                     | 

Признаками типа:

|                 |  Электрическая проводимость - average ? |  Электрическая проводимость - good ? |  Электрическая проводимость - low ? |
|-----------------|:-----------------:|:---------------:|:--------------:|
|**Образец пыли 1**  |       Да  (1)   |      Нет (0)  |      Нет (0) |
|**Образец пыли 2**  |       Нет (0)   |      Да  (1)  |      Нет (0) |
|**Образец пыли 3**  |       Нет (0)   |      Нет (0)  |      Да  (1) |

Таким образом, наши новые признаки говорят относится ли oбразец пыли к определённому типу электрической проводимости или нет. При таком подходе для каждого категориального признака появляется столько новых колонок, сколько есть возможных категорий. Одна из колонок будет заполнена `1`, а остальные `0`.

In [ ]:
dummies = pd.get_dummies(data=df, columns=text_features)

В функцию **get_dummies()** в качестве параметров передаём:
*  **data** - исходные данные
*  **columns** - имена колонок, в которых находятся категориальные признаки

In [ ]:
dummies.head()

Как видим, после кодирования у нас получилось на 5 признаков больше (14 вместо 9).

In [ ]:
df.shape

In [ ]:
dummies.shape

Второй способ кодирования категориальных признаков заключается в том, чтобы просто присвоить каждой категории уникальный номер, и заменить значения признаков на соответствующие номера.

Например, вместо таблицы:

|                 |    Электрическая проводимость |  
|-----------------|:-------------------------------:|
|**Образец пыли 1**  |       average                   | 
|**Образец пыли 2**  |       average                     |
|**Образец пыли 3**  |       good                     |
|**Образец пыли 4**  |       low                   |
|**Образец пыли 5**  |       average                      |
|**Образец пыли 6**  |       good                     | 

Получаем:

|                 |    Электрическая проводимость |  
|-----------------|:-------------------------------:|
|**Образец пыли 1**  |       1                       | 
|**Образец пыли 2**  |       1                       | 
|**Образец пыли 3**  |       2                       |
|**Образец пыли 4**  |       3                       |
|**Образец пыли 5**  |       1                       |
|**Образец пыли 6**  |       2                       |

Здесь все значения **average** заменены на `1`, **good** заменены на `2`, **low** заменены на `3`.

In [ ]:
from sklearn.preprocessing import LabelEncoder

Сперва создаем прототип кодировщика: 

In [ ]:
label_encoder = LabelEncoder()

Так как нам нужно закодировать сразу список признаков, мы будем делать это в цикле. Рассматриваем каждый текстовый признак из списка *text_features*, далее методу **fit_transform()** передаем в качестве аргумента этот признак


К полученным числовым представлениям признака будем прибавлять единичку, чтобы кодирование начиналось с 1, а не с 0. Затем результат будем записывать в табличку. 

In [ ]:
for col in text_features:
    df[col] = label_encoder.fit_transform(df[col]) + 1

Посмотрим на данные теперь:

In [ ]:
df.head()

In [ ]:
df.shape

При таком способе кодирования количество признаков не меняется.

**Чем может быть обусловлен выбор метода кодирования признаков?**

## 3. Построить регрессионную модель (классический вариант)

Используя средства библиотеки **statsmodels** построим модель множественной линейной регрессии УЭС на остальные предикторы и проанализируем полученный результат.

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

results = smf.ols('resistivity ~ temperature + humidity + density + dust_capacity + particle_size + conductivity + dust_dispersiveness + formation', data=df).fit()
results.summary()

### Препарируем OLS model


`model = sm.ols(formula="a ~ b + c", data=df).fit()`

`result.summary()`

В левой части первой таблицы представлена основная информация об обучаемой модели:

| Элемент                    | Описание                                                                          |
|:--------------------------:|:---------------------------------------------------------------------------------:|
| Dep. Variable              | Какая переменная является ответом в модели                                        |
| Model                      | Какую модель вы используете                                                       | 
| Method                     | Как рассчитывались параметры модели                                               | 
| No. Observations           | Количество наблюдений                                                             | 
| DF Residuals               | Степени свободы остатков = количество наблюдений - количество параметров модели   |
| DF Model                   | Количество параметров в модели (не включая постоянный член, если он присутствует) |

Правая часть первой таблицы показывает качество обученной модели:

| Элемент                    | Описание                                                                          |
|:--------------------------:|:---------------------------------------------------------------------------------:|
| R-squared              | Коэффициент детерминации. Статистическая мера того, насколько хорошо линия регрессии аппроксимирует реальные точки данных.|
| Adj. R-squared                     | Приведенное выше значение скорректированное на основе количества наблюдений и степеней свободы остатков.| 
| F-statistic                    | Мера, насколько значима полученная модель. Среднеквадратическая ошибка модели, деленная на среднеквадратичную ошибку остатков| 
| Prob (F-statistic)           | Вероятность того, что вы получите приведенную выше статистику или еще более эксремальную, учитывая нулевую гипотезу о том, что модель незначима (p-value)| 
| Log-likelihood               | Значение логарифма функции правдоподобия |
| AIC                   | Информационный критерий Акаике. Корректирует логарифмическую вероятность на основе количества наблюдений и сложности модели. |
| BIC                   | Байесовский информационный критерий. Аналогичен AIC, но имеет более высокий штраф для моделей с большим количеством параметров.|

Вторая таблица представляет информацию по каждому из коэффициентов

| Элемент                    | Описание                                                                          |
|:--------------------------:|:---------------------------------------------------------------------------------:|
|             | Название переменной в модели|
| coef | Расчетное значение коэффициента| 
| std err| Средняя квадратическая ошибка оценки коэффициента| 
|t| Значение t-статистики. Это показатель того, насколько статистически значим коэффициент.| 
|$$ P > |t|$$| P-value для проверки нулевой гипотезы о том, что коэффициент = 0. Если меньше уровня значимости, это указывает на наличие статистически значимой связи между переменной и ответом|
| 95.0% Conf. Interval| Нижнее и верхнее значения 95% доверительного интервала для коэффициента |

В последней части таблицы приводятся несколько статистических тестов для оценки распределения остатков.

| Элемент                    | Описание                                                                          |
|:--------------------------:|:---------------------------------------------------------------------------------:|
|Skewness| Мера симметрии данных относительно среднего. Нормально распределенные ошибки должны быть симметрично распределены относительно среднего (равные суммы выше и ниже линии)|
|Kurtosis| Мера формы распределения. Сравнивает количество данных, близких к среднему, с данными, далекими от среднего (в хвостах)| 
|Omnibus| Тест Д’Агостино. Он обеспечивает комбинированный статистический тест на отличие асимметрии и эксцесса от нормального| 
|Prob(Omnibus)| P-value для теста Д’Агостино, нулевая гипотеза - остатки распределены нормально| 
|Jarque-Bera| Тест Харке—Бера на нормальность по значениям асимметрии и эксцесса|
|Prob (JB)| P-value для теста Харке—Бера, нулевая гипотеза - остатки распределены нормально |
|Durbin-Watson| Тест на наличие автокорреляции (ошибки не являются независимыми). В случае отсутствия автокорреляции значение DW=2. При положительной автокорреляции DW стремится к 0, а при отрицательной — к 4:|
|Cond. No| Тест на мультиколлинеарность. Если превышает 30, регрессия может иметь сильную мультиколлинеарность. |

Как по $p-value$ определить, есть ли основания отвергнуть нулевую гипотезу. 

1. Зафиксируем уровень значимости $\alpha$ – это вероятность отвергнуть нулевую гипотезу при условии, что она верна.
2. $p-value$ – это минимальный уровень значимости, на котором нулевая гипотеза может быть отвергнута.

Значит, если:

$p−value < \alpha$ $\Rightarrow$ $H_0$ отвергаем на уровне значимости $\alpha$ (на имеющихся данных)

$p−value \geq \alpha$ $\Rightarrow$ $H_0$ не отвергаем на уровне значимости $\alpha$ (на имеющихся данных)

## 4. Провести анализ полученной модели
**АНАЛИЗИРУЕМ ЭТО ;)**

**Немного магии или алхимии анализа данных**

Вспомним, как выглядит распределение целевой переменной.

In [ ]:
df.resistivity.hist(figsize=(5, 5))

Давайте преобразуем нашу целевую переменную прологарифмировав значения. 

In [ ]:
import numpy as np
df['lnR']=np.log(df['resistivity'])

In [ ]:
df.lnR.hist(figsize=(5, 5))

Построим новую модель, теперь уже для преобразованного значения целевой переменной.

In [ ]:
results_2 = smf.ols('lnR ~ temperature + humidity + density + dust_capacity + particle_size + conductivity + dust_dispersiveness + formation', data=df).fit()
results_2.summary()

**Стало ли лучше?**

Тогда давайте добавим еще несколько признаков, используя исходные столбцы.

In [ ]:
df['temperature2']=df['temperature']**2
df['humidityIn']=1/df['humidity']
df['Lndensity']=np.log(df['density'])

Посмотрим на таблицу корреляций.

In [ ]:
sns.heatmap(df.corr(), cbar_kws= {'orientation': 'vertical'}, annot=True, fmt='.1f', vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')

И построим обновленную модель.

In [ ]:
results_3 = smf.ols('lnR ~ temperature2 + temperature + humidity + humidityIn + density + Lndensity + dust_capacity + particle_size + conductivity + dust_dispersiveness + formation', data=df).fit()
results_3.summary()

Анализируйя предыдущий результат удалим признаки, коэффициенты перед которыми статистически незначимы.

In [ ]:
results_4 = smf.ols('lnR ~ temperature + humidity + humidityIn + density + Lndensity + conductivity + formation', data=df).fit()
results_4.summary()

Итоговая формула модели выглядит так

$$lnR = -0.0428\cdot temperature -0.1909\cdot humidity + 15.7884\cdot humidityIn -0.0146\cdot density+ 14.0062\cdot Lndensity + 1.2576\cdot conductivity -1.4192\cdot formation -79.2692$$

А если вспомнить, что задача стояла в предсказаний значения `resistivity`, необходимо провести обратное преобразование, чтобы перейти от `lnR` к `resistivity`.

$$resistivity = e^{-0.0428\cdot temperature -0.1909\cdot humidity + 15.7884\cdot humidityIn -0.0146\cdot density+ 14.0062\cdot Lndensity + 1.2576\cdot conductivity -1.4192\cdot formation -79.2692}$$

**А какие еще изменения модели (или предобработки данных) можно протестировать, чтобы улучшить качество предсказаний?**

### Проверка допущений применения классической модели линейной регрессии

###### Допущение 1. Ошибка имеет центрированное распределение

Построим $qq$-график и гистограмму ошибок(остатков модели) для визуальной проверки.

In [ ]:
import scipy as sc
plt.figure(figsize=(16,7))
plt.subplot(121)
sc.stats.probplot(results_4.resid, dist="norm", plot=plt)
plt.subplot(122)
np.log(results_4.resid).plot.hist()
plt.xlabel('Остатки модели', fontsize=14)
plt.show()

Проверим гипотезу о том, что среднее значение ошибок $\mu = 0$.

**$t$-тест Стьюдента** 

$H_0$:$E(X)=\mu$.

$H_1$:$H_0$ - неверна.

In [ ]:
from scipy import stats
print("Критерий Стьюдента: p-value=%f" % stats.ttest_1samp(results_4.resid, 0)[1])

In [ ]:
results_4.resid.mean()

Среднее значение равно $\mu = 0$, однако по $qq$-графику и гистограмме видно, что данные явно не являются нормально распределенными. 

Проверим это статситическим критерием.

**Тест Лиллиефорса** 

$H_0$:распределение нормальное.

$H_1$:распределение не нормальное.

In [ ]:
from statsmodels.stats.diagnostic import lilliefors
print("Критерий Лиллиефорса, p-value: %f" % lilliefors(results_4.resid, dist='norm')[1])

**Первое допущение нарушается.**

###### Допущение 2. Гомоскедастичность регрессионных остатков и отсутствие автокорреляции.

**Тест Бройша — Пагана**  — один из статистических тестов для проверки наличия гетероскедастичности случайных ошибок регрессионной модели. В данном тесте проверяется линейная зависимость дисперсии случайных ошибок от некоторого набора переменных. Если статистика теста имеет значение $p-value$ ниже соответствующего уровня значимости, то нулевая гипотеза о гомоскедастичноости отвергается.

In [ ]:
import statsmodels.stats.api as sms
print('Тест Бройша — Пагана:  p-value=%f' % sms.het_breuschpagan(results_4.resid, results_4.model.exog)[1])

**Критерий Дарбина—Уотсона** — статистический критерий, используемый для тестирования автокорреляции первого порядка элементов исследуемой последовательности. В случае отсутствия автокорреляции статистика критерия равна 2, при положительной автокорреляции статистика стремится к нулю, а при отрицательной — к 4.

In [ ]:
from statsmodels.stats.stattools import durbin_watson
durbin_watson(results_4.resid)

In [ ]:
plt.scatter(results_4.predict(),  results_4.resid,
            c='blue', marker='o', label='Training data')
plt.xlabel('Предсказания модели')
plt.ylabel('Остатки')
plt.hlines(y=0, xmin=-6, xmax=15,  color='red')
plt.show()

**Второе допущение нарушается.**

###### Допущение 3. Предикторы не должны быть связаны функциональной линейной зависимостью.

In [ ]:
sns.heatmap(df[['temperature', 'humidity','humidityIn','density', 'Lndensity', 'conductivity', 'formation']].corr(), cbar_kws= {'orientation': 'vertical'}, annot=True, fmt='.1f', vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')
plt.show()

Функциональная линейная зависимость наблюдается у переменнных `density` и `Lndensity` - значит из этих двух переменных необходимо оставить только одну(ту, которая сильнее связана с откликом). Также видна сильная линейная зависимость между переменными `humidity` и `humidityIn`.

**Третье допущение нарушается.**

###### Допущение 4. Объем выборки должен быть значительно больше числа признаков.

Выборка $m=360 \gg n=7$

**Четвертое допущение выполняется.**

**Ну хоть что-то ;)**

**Вывод:** при нарушении допущений классической линейной регрессии мы все еще можем воспользоваться моделью для прогнозирования, однако необходимо учитывать следующее:
   - доверительные интервалы, полученные для коэффициентов модели, могут быть ненадежными;
   - при построении модели были учтены не все зависимости в данных, поэтому модель можно улучшить.
   
Коэффициент детерминации  $R^2$ итоговой модели равен $0.707$.

Получим предсказание нашей модели для произвольного образца пыли (придумаем его сами).

In [ ]:
df_predict = pd.DataFrame({'temperature': [20],
                           'humidity': [1.4],
                           'humidityIn': [0.71],
                           'density': [1158],
                           'Lndensity': [7.05],
                           'conductivity': [1],
                           'formation': [2]})
df_predict

In [ ]:
results_4.predict(df_predict)

In [ ]:
np.exp(results_4.predict(df_predict))

# Машинное обучение (начало)

## 5. Разделить данные для обучения и тестирования

Разделение исходных данных на обучающий и тестовый набор используется для оценки производительности алгоритмов машинного обучения.

Набор обучающих данных используется для обучения модели, все ответы целевой переменной для этого набора нам известны. Второй набор называется набором тестовых данных, этот набор используется исключительно для прогнозов.

Библиотека `scikit-learn` предоставляет нам модуль `model_selection`, в котором у нас есть функция разделения `train_test_split()`.


```python
train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
```

Параметры:

- arrays: входные данные, такие как списки, массивы, фреймы данных или матрицы.
- test_size: это вещественное число, значение которого находится в диапазоне от 0.0 до 1.0, оно представляет собой долю нашего размера тестовых данных.
- train_size: то вещественное число, значение которого находится в диапазоне от 0.0 до 1.0, оно представляет собой долю нашего размера обучающих данных.
- random_state: этот параметр используется для управления перемешиванием, применяемой к данным перед применением разделения.
- shuffle: этот параметр используется для перетасовки данных перед разделением, его значение по умолчанию равно true.
- stratify: этот параметр используется для разделения данных стратифицированным образом.

In [ ]:
from sklearn import model_selection

In [ ]:
dataset_train, dataset_test = model_selection.train_test_split(df, test_size=0.3, random_state=42)

In [ ]:
dataset_train

In [ ]:
dataset_test

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df.drop(columns = ['resistivity', 'lnR']), df['lnR'], 
                                                    test_size=0.3, 
                                                    random_state=42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

**Выбираем метод, который будем использовать**

Проще всего начать с простых методов. 
* Линейная регрессия *linear regression*

На выбор метода для построения модели влияет набор признаков, размер выборки, интуиция про то, какая связь между входными переменными и целевой. Но часто решение принимается исходя из того, какая модель сработала лучше.

Для корректной работы с методами построения моделей в python требуется загрузить специальную библиотеку
**sklearn**, программную библиотеку на языке python для для машинного обучения и анализа данных.

Мы импортируем модул для линейных моделей из этой библиотеки:
 * *linear_model* - тут находятся все линейные модели


In [ ]:
from sklearn import linear_model

Прежде чем начать делать ремонт, нужно подготовить инструменты для работы. Аналогично в нашем случае, прежде чем обучать модели, нужно создать их прототипы.  

Чтобы создать модель линейной регресии, пишем имя модуля 'linear_model', затем точку, затем название модели.

Для этого нужно выполнить следующий код:

```python
linear_regression_model = linear_model.LinearRegression()
linear_regression_model
```

Результат выполнения должен быть следующим:

```python
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
```
 или
 
```python
LinearRegression()
```

In [ ]:
linear_regression_model = linear_model.LinearRegression() # создаем модель

In [ ]:
linear_regression_model # смотрим, что получилось

## 6. Обучить модель на обучающей выборке

Теперь, когда мы создали прототипы обеих моделей, можем их обучить с помощью обучающей выборки. 

Для этого вызываем метод **fit()** у каждой модели и передаем ему на вход два аргумента: 
таблицу входных признаков и столбец значений целевой переменной - (X_train, y_train)

In [ ]:
linear_regression_model.fit(X_train, y_train)

In [ ]:
X_train.columns

In [ ]:
# Выведем параметры модели в явном виде
print(linear_regression_model.intercept_, linear_regression_model.coef_)

* Мы получили обученную модель. 
* Теперь необходимо провалидировать модель на новых(которых модель не видела) тестовых данных. 

## 7. Провалидировать модель на тестовой выборке

Получим прогнозы целевой переменной на тестовых данных для модели линейной регрессии. 

Для этого вызовем у каждой модели метод **predict()**, в качестве аргумента передадим *X_test*.

In [ ]:
test_predictions_linear = linear_regression_model.predict(X_test)

Качество регрессионной модели оценим двумя способами: 
1. Сравним визуально прогнозы с настоящими значениями (тестовые с предсказанием)
2. Сравним метрики качества

Визуализируем прогноз линейной модели и настоящие значения из тестовой выборки.

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(y_test, test_predictions_linear) # рисуем точки, соответствущие парам настоящее значение - прогноз
plt.plot([-5, 15], [-5, 15]) # рисуем прямую, на которой предсказания и настоящие значения совпадают
plt.xlabel('Настоящее ln УЭС', fontsize=20)
plt.ylabel('Предсказанное ln УЭС', fontsize=20);

Вычислим **метрики качества регрессионной модели**

Для корректного подсчета метрик качества модели в python требуется загрузить их из библиотеки **sklearn**. 

Мы используем три метрики качества:
 * *mean_absolute_error* - средняя абсолютная ошибка $MAE=\sum\limits_{i=1}^{n}\frac{|y_i - \hat{y}_i|}{n}$
 * *mean_squared_error* - средняя квадратичная ошибка $MSE=\sum\limits_{i=1}^{n}\frac{(y_i - \hat{y}_i)^2}{n}$
 * *r2_score* - коэффициент детерминации $R^2={\frac{\sum\limits_{i=1}^{n}(\hat{y}_i-\bar{y})^2}{\sum\limits_{i=1}^{n}(y_i-\bar{y})^2}}={1-\frac{\sum\limits_{i=1}^{n}(y_i-\hat{y}_i)^2}{\sum\limits_{i=1}^{n}(y_i-\bar{y})^2}}$

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Подсчитаем ошибки для линейной модели.

Для этого вызовем методы **mean_absolute_error()**, **mean_squared_error()** и **r2_score()**. На вход им передается столбец настоящих значений *y_test* и столбец значений, предсказанных моделью линейной регрессии *test_predictions_linear*.

In [ ]:
mean_absolute_error_linear_model = mean_absolute_error(y_test, test_predictions_linear) 
mean_squared_error_linear_model = mean_squared_error(y_test, test_predictions_linear)
r2_score_linear_model = r2_score(y_test, test_predictions_linear)

Теперь напечатаем полученные ошибки. Обычно смотрят на корень из среднеквадратичной ошибки, RMSE. Чтобы извлечь корень нам понадобится библиотека **Numpy**. С помощью неё можно быстро производить вычисления сразу над массивами чисел

In [ ]:
import numpy as np
# %7.2f - вывод числа с плавающей точкой (f - float/double); 7 - до скольки дополнять пробелами, .2 - сколько символов после запятой

print("MAE: {0:7.2f}, RMSE: {1:7.2f}, R2: {2:7.2f} для линейной модели".format(
        mean_absolute_error_linear_model, 
        np.sqrt(mean_squared_error_linear_model),
        r2_score_linear_model))